# Lab 4: Belief Propagation

This lab is built around the process of identifying the fault with a coffee machine using the least amount of effort.

Your task is to:
 1. Given the structure of a graphical model for the state of a coffee machine learn the distributions from data.
 2. Implement belief propagation, so you can evaluate the probability of each failure given the available evidence.
 3. Use Bayesian decision theory to generate a diagnostic flow chart that will identify the problem as quickly as possible on average, taking account of how long it takes to perform each observation.

## Marking and Submission

These lab exercises are marked, and contribute to your final grade. For this lab exercise there are 3 places where you are expected to enter your own code, for 30 marks overall. Every place you have to add code is indicated by

`# **************************************************************** n marks`

with instructions above the code block.

Please submit your completed workbook using Moodle before 5pm on the 29th November 2017. The workbook you submit must be an `.ipynb` file, which is saved into the directory you're running Jupyter; alternatively you can download it from the menu above using `File -> Download As -> Notebook (.ipynb)`. Remember to save your work regularly (Save and checkpoint in the File menu, the icon of a floppy disk, or Ctrl-S); the version you submit should have all code blocks showing the results (if any) of execution below them. You will normally receive feedback within a week.

In [1]:
%matplotlib inline

import zipfile
import io
import csv

import numpy
import matplotlib.pyplot as plt

## Coffee Machine Data Set

You can download from moodle a zip file that contains the data as a csv file. The below code will load the data directly from the zip file, so you don't have to unzip it.

Each row of the file contains a fully observed coffee machine, with the state of every random variable. The random variables are all binary, with False represented by 0 and True represented by 1. The variables are:

Failures (you're trying to detect these):  
&nbsp; &nbsp; 0. `he` - Have mains electricity  
&nbsp; &nbsp; 1. `fp` - Fried power supply unit  
&nbsp; &nbsp; 2. `fc` - Fried circuit board  
&nbsp; &nbsp; 3. `wr` - Water in reservoir  
&nbsp; &nbsp; 4. `dp` - Dead pump  
&nbsp; &nbsp; 5. `fh` - Fried heating element  
&nbsp; &nbsp; 6. `gs` - Group head gasket forms seal  

Mechanism (these are unobservable):  
&nbsp; &nbsp; 7. `pw` - Power supply unit works  
&nbsp; &nbsp; 8. `cb` - Circuit board works  
&nbsp; &nbsp; 9. `gw` - Get water out of group head  
&nbsp; &nbsp; 10. `hw` - Get hot water out of group head  
&nbsp; &nbsp; 11. `li` - Leaks during infusion  

Diagnostic (these are the tests the mechanic can run - observable):  
&nbsp; &nbsp; 12. `ls` - Room lights switch on  
&nbsp; &nbsp; 13. `vp` - A voltage is measured across power supply unit  
&nbsp; &nbsp; 14. `bs` - Burning smell  
&nbsp; &nbsp; 15. `lo` - Power light switches on  
&nbsp; &nbsp; 16. `hp` - Can hear pump  
&nbsp; &nbsp; 17. `me` - Makes espresso  
&nbsp; &nbsp; 18. `ta` - Makes a hot, tasty espresso  

For the above the number is the column number of the provided data (`dm`) and the two letter code a suggested variable name.

For if you are unfamiliar with a espresso coffee machine here is a brief description of how one works:
> The user puts ground coffee into a portafilter (round container with a handle and two spouts at the bottom), tamps it (compacts the coffee down), and clamps the portafilter into the group head at the front of the machine. A gasket (rubber ring) forms a seal between the portafilter and group head. A button is pressed. Water is drawn from a reservoir by a pump into a boiler. In the boiler a heating element raises the waters temperature, before the pump pushes it through the group head and into the portafilter at high pressure. The water passes through the coffee grinds and makes a tasty, tasty espresso.

The graphical model showing how the variables are related is included on moodle as `coffee machine.pdf`; here it is given as conditional probabilities:

 * $P(\texttt{have electricity})$
 * $P(\texttt{fried psu})$
 * $P(\texttt{fried circuit board})$
 * $P(\texttt{water in reservoir})$
 * $P(\texttt{dead pump})$
 * $P(\texttt{fried heating element})$
 * $P(\texttt{group head gasket forms seal})$
 
 * $P(\texttt{psu works}\enspace|\enspace\texttt{have electricity},\enspace\texttt{fried psu})$
 * $P(\texttt{circuit board works}\enspace|\enspace\texttt{psu works},\enspace\texttt{fried circuit board})$
 * $P(\texttt{get water}\enspace|\enspace\texttt{circuit board works},\enspace\texttt{water in reservoir},\enspace\texttt{dead pump})$
 * $P(\texttt{get hot water}\enspace|\enspace\texttt{get water},\enspace\texttt{fried heating element})$
 * $P(\texttt{leaks during infusion}\enspace|\enspace\texttt{get water},\enspace\texttt{group head gasket forms seal})$

 * $P(\texttt{lights switch on}\enspace|\enspace\texttt{have electricity})$
 * $P(\texttt{voltage across psu}\enspace|\enspace\texttt{psu works})$
 * $P(\texttt{burning smell}\enspace|\enspace\texttt{fried psu},\enspace\texttt{fried circuit board})$ 
 * $P(\texttt{power light on}\enspace|\enspace\texttt{circuit board works})$
 * $P(\texttt{can hear pump}\enspace|\enspace\texttt{circuit board works},\enspace\texttt{dead pump})$
 * $P(\texttt{makes espresso}\enspace|\enspace\texttt{get water},\enspace\texttt{group head gasket forms seal})$
 * $P(\texttt{tasty}\enspace|\enspace\texttt{makes espresso},\enspace\texttt{get hot water})$

Note that while the model is close to what you may guess the probabilities are not absolute, to account for mistakes and unknown failures. For instance, the mechanic may make a mistake while brewing an espresso and erroneously conclude that the machine is broken when it is in fact awesome. The probabilities associated with each failure are not uniform.

In [2]:
# It may prove helpful to have a mapping between the suggested variable names and column indices...
nti = dict() # 'name to index'

nti['he'] = 0
nti['fp'] = 1
nti['fc'] = 2
nti['wr'] = 3
nti['dp'] = 4
nti['fh'] = 5
nti['gs'] = 6

nti['pw'] = 7
nti['cb'] = 8
nti['gw'] = 9
nti['hw'] = 10
nti['li'] = 11

nti['ls'] = 12
nti['vp'] = 13
nti['bs'] = 14
nti['lo'] = 15
nti['hp'] = 16
nti['me'] = 17
nti['ta'] = 18

# For conveniance this code loads the data from the zip file,
# so you don't have to decompress it (takes a few seconds to run)...
with zipfile.ZipFile('coffee_machines.zip') as zf:
    with zf.open('coffee_machines.csv') as f:
        sf = io.TextIOWrapper(f)
        reader = csv.reader(sf)
        next(reader)
        dm = []
        for row in reader:
            dm.append([int(v) for v in row])
        dm = numpy.array(dm, dtype=numpy.int8)

print('Data: {} exemplars, {} features'.format(dm.shape[0], dm.shape[1]))
print('     Broken machines  =', dm.shape[0] - dm[:,nti['ta']].sum())
print('     Working machines =', dm[:,nti['ta']].sum())


Data: 262144 exemplars, 19 features
     Broken machines  = 113637
     Working machines = 148507


## 1. Learn Model

Below a set of variables to represent conditional probability distributions have been defined. They are a Bernoulli trial for each combination of conditional variables, given as $P(\texttt{False}|...)$ in `[0,...]` and $P(\texttt{True}|...)$ in `[1,...]` (It may be easier to think of them as boring categorical distributions).

To fit a maximum likelihood model you should first use them as counters - loop over the data set and count how many of each combination exist. You must then normalise them so that the sum over axis 0 is always 1. There is an extra mark for doing the right thing and including a prior. You may want to know that the conjugate prior to the Bernoulli trial represented as $\left[P(\texttt{False}), P(\texttt{True})\right]$ is a Dirichlet distribution; a uniform prior would be a reasonable choice (it can be argued that the expected failure probability is low, and you should adjust the first 7 variables accordingly, but given the quantity of data available it's not going to matter).

Hint:
 * The use of `0=False` and `1=True` both in the dm table and in the conditional probability distributions is very deliberate.
 * Consider putting all of the variables into a list with extra information about them (such as indices from `nti`) to make your code neater.
 * If you make a mistake you could easily end up with NaN or infinity - which would break the next step. Print them out so you can check they are valid!

__(5 marks)__
 * 3 marks for counting all conditions.
 * 1 mark for normalising distributions.
 * 1 mark for adding a sensible prior.

In [3]:
# A set of variables that will ultimately represent conditional probability distributions.
# The naming convention is that P_he means P(he), or that P_pw_he_fp means P(pw|he,fp)...

P_he          = numpy.zeros(2)
P_fp          = numpy.zeros(2)
P_fc          = numpy.zeros(2)
P_wr          = numpy.zeros(2)
P_dp          = numpy.zeros(2)
P_fh          = numpy.zeros(2)
P_gs          = numpy.zeros(2)

P_pw_he_fp    = numpy.zeros((2,2,2))
P_cb_pw_fc    = numpy.zeros((2,2,2))
P_gw_cb_wr_dp = numpy.zeros((2,2,2,2))
P_hw_gw_fh    = numpy.zeros((2,2,2))
P_li_gw_gs    = numpy.zeros((2,2,2))

P_ls_he       = numpy.zeros((2,2))
P_vp_pw       = numpy.zeros((2,2))
P_bs_fp_fc    = numpy.zeros((2,2,2))
P_lo_cb       = numpy.zeros((2,2))
P_hp_cb_dp    = numpy.zeros((2,2,2))
P_me_gw_gs    = numpy.zeros((2,2,2))
P_ta_me_hw    = numpy.zeros((2,2,2))



# **************************************************************** 5 marks
indices = ['he','fp','fc','wr','dp','fh','gs','pw','cb','gw','hw','li','ls','vp','bs','lo','hp','me','ta'];
count = dm.shape[0];

def prob(index):
    
    p = numpy.zeros(2)
    p[0] = (dm.shape[0] - dm[:,nti[index]].sum())/dm.shape[0]
    p[1]= (dm[:,nti[index]].sum())/dm.shape[0]
    return p

def conditional1(index, con1):
    p = numpy.zeros((2,2))
    for i in range(0,2):
        data = dm[dm[:,nti[con1]]==i]
        p[0][i] = (data.shape[0] - data[:,nti[index]].sum())/data.shape[0]
        p[1][i] = (data[:,nti[index]].sum())/data.shape[0]
                    
    return p
    
def conditional2(index, con1, con2):
    p = numpy.zeros((2,2,2));
    for i in range(0,2):
        data = dm[dm[:,nti[con1]]==i]
        for j in range(0,2):
            data2 = data[data[:,nti[con2]]==j]
            p[0][i][j] = (data2.shape[0] - data2[:,nti[index]].sum())/data2.shape[0]
            p[1][i][j] = (data2[:,nti[index]].sum())/data2.shape[0]
            
    return p

def conditional3(index, con1, con2, con3):
    p = numpy.zeros((2,2,2,2));
    for i in range(0,2):
        data = dm[dm[:,nti[con1]]==i]
        for j in range(0,2):
            data2 = data[data[:,nti[con2]]==j]
            for k in range(0,2):
                data3 = data2[data2[:,nti[con3]]==k]
                p[0][i][j][k] = (data3.shape[0] - data3[:,nti[index]].sum())/data3.shape[0]
                p[1][i][j][k] = (data3[:,nti[index]].sum())/data3.shape[0]
    
    return p

P_he = prob('he')
P_fp = prob('fp')
P_fc = prob('fc')
P_wr = prob('wr')
P_dp = prob('dp')
P_fh = prob('fh')
P_gs = prob('gs')

P_pw_he_fp    = conditional2('pw','he','fp')
P_cb_pw_fc    = conditional2('cb','pw','fc')
P_gw_cb_wr_dp = conditional3('gw','cb','wr','dp')
P_hw_gw_fh    = conditional2('hw','gw','fh')
P_li_gw_gs    = conditional2('li','gw','gs')

P_ls_he       = conditional1('ls','he')
P_vp_pw       = conditional1('vp','pw')
P_bs_fp_fc    = conditional2('bs','fp','fc')
P_lo_cb       = conditional1('lo','cb')
P_hp_cb_dp    = conditional2('hp','cb','dp')
P_me_gw_gs    = conditional2('me','gw','gs')
P_ta_me_hw    = conditional2('ta','me','hw')

print(P_dp)
print (P_hw_gw_fh)
print (P_lo_cb)

print(P_gw_cb_wr_dp)
print(P_ta_me_hw)



[ 0.94974518  0.05025482]
[[[ 1.  1.]
  [ 0.  1.]]

 [[ 0.  0.]
  [ 1.  0.]]]
[[  1.00000000e+00   9.75109058e-04]
 [  0.00000000e+00   9.99024891e-01]]
[[[[ 1.  1.]
   [ 1.  1.]]

  [[ 1.  1.]
   [ 0.  1.]]]


 [[[ 0.  0.]
   [ 0.  0.]]

  [[ 0.  0.]
   [ 1.  0.]]]]
[[[ 1.          1.        ]
  [ 1.          0.19794446]]

 [[ 0.          0.        ]
  [ 0.          0.80205554]]]


### Brute Force
The below code does exactly what you want to implement for step 2, but it brute forces it. Slow and memory efficient of course, but lets you test it. Also means that if you can't solve question 2 you can still use this to have a go at question 3.

In [4]:
def brute_marginals(known):
    """known is a dictionary, where a random variable index existing as a key in the dictionary
    indicates it has been observed. The value obtained using the key is the value the
    random variable has been observed as. Returns a 19x2 matrix, such that [rv,0] is the
    probability of random variable rv being False, [rv, 1] the probability of being True."""
    
    # Calculate the full joint (please don't ask)...
    dims = 'abcdefghijklmnopqrs'
    joint = numpy.einsum('a,b,c,d,e,f,g,hab,ihc,jide,kjf,ljg,ma,nh,obc,pi,qie,rjg,srk->' + dims,
                         P_he, P_fp, P_fc, P_wr, P_dp, P_fh, P_gs,
                         P_pw_he_fp, P_cb_pw_fc, P_gw_cb_wr_dp, P_hw_gw_fh, P_li_gw_gs,
                         P_ls_he, P_vp_pw, P_bs_fp_fc, P_lo_cb, P_hp_cb_dp, P_me_gw_gs, P_ta_me_hw)
    
    # Multiply in the known states (zero out the dimensions we know it's not)...
    for key, value in known.items():
        other = abs(value-1)
        index = [slice(None)] * len(joint.shape)
        index[key] = other
        joint[tuple(index)] = 0.0
    
    # Calculate and return all marginals...
    ret = numpy.empty((len(joint.shape), 2))
    for row in range(ret.shape[0]):
        ret[row,:] = numpy.einsum(dims + '->' + dims[row], joint)
    
    ret /= ret.sum(axis=1)[:,None]
    return ret


## 2. Belief Propagation

Your task is to write a function that take known states and calculates the marginal probability for every state of the graphical model. This will require using the _sum-product algorithm_ and passing all messages on the graph.

Here is the wikipedia page for reference: https://en.wikipedia.org/wiki/Belief_propagation (not the greatest, but not horrible)

Hints:
 * It is strongly suggested to use the same calling convention as `brute_marginals` above for your final interface.
 * The order of the variables above is such that each variable is dependent only on variables that occur before it. Depending on your approach this information may save you some effort.
 * You will want to add code before the function to prepare. This might involve creating Factor and Node classes. Or a list of edges in the graphical model, in the order they need to be processed. Choose whatever you are comfortable with, but spend time thinking it through first. If you get it wrong your code will get messy!
 * This problem is small enough that you can brute force it - code to do so has been provided above. Do test that your implementation is correct by comparing them!

__(15 marks)__
 * 5 marks for creating sensible data structures.
 * 7 marks for sending the messages correctly.
 * 2 marks for correctly calculating the marginals at the end.
 * 1 mark for testing.

In [21]:
ret = brute_marginals({nti['ta'] : 1})

ret1 = numpy.empty(ret.shape)

#Forward Messages
"""
P_pw = numpy.einsum('abc,b,c->a', P_pw_he_fp, P_he,P_fp)
P_pw1 = numpy.einsum('abc,c,b->a', P_pw_he_fp, P_fp,P_he)
other1 = P_he[None,:,None]
other2 = P_fp[None,None,:]
P_pw_Joint = P_pw_he_fp * other1 * other2
P_pw3 = P_pw_Joint * P_pw[:,None,None] * other2
P_cb = numpy.einsum('abc,b,c->a',P_cb_pw_fc,P_pw,P_fc)
P_cb /= P_cb.sum()
P_gw = numpy.einsum('abcd,b,c,d->a',P_gw_cb_wr_dp,P_cb,P_wr,P_dp)
P_gw /= P_gw.sum()
P_hw = numpy.einsum('abc,b,c->a',P_hw_gw_fh,P_gw,P_fh)
P_hw /= P_hw.sum()
P_li = numpy.einsum('abc,b,c->a',P_li_gw_gs,P_gw,P_gs)
P_li /= P_li.sum()


P_ls = numpy.einsum('ab,b->a',P_ls_he,P_he)
P_ls_Joint = P_ls_he * P_he[None,:]
P_ls /= P_ls.sum()
P_vp = numpy.einsum('ab,b->a',P_vp_pw,P_pw)
P_vp /= P_vp.sum()
P_bs = numpy.einsum('abc,b,c->a', P_bs_fp_fc, P_fp,P_fc)
P_bs /= P_bs.sum()
P_lo = numpy.einsum('ab,b->a',P_lo_cb,P_cb)
P_lo /= P_lo.sum()
P_hp = numpy.einsum('abc,b,c->a', P_hp_cb_dp, P_cb,P_dp)
P_hp /= P_he.sum()
P_me = numpy.einsum('abc,b,c->a', P_me_gw_gs, P_gw,P_gs)
P_me /= P_me.sum()
P_ta = numpy.einsum('abc,b,c->a', P_ta_me_hw, P_me,P_hw)
P_ta /= P_ta.sum()


print(P_pw)
print(P_pw1)

"""

# **************************************************************** 15 marks

class RV:
    """Defines a random variable, which is really just storage for messages
    both arrviing and leaving from it."""
    
    def __init__(self, links):
        """Links is the number of links the RV has."""
        # Storage for the messages from/to each of the links...
        self.msg_in = [None] * links
        self.msg_out = [None] * links
        self.known_Belief = numpy.ones(2)


    def calc_msg(self,link):
        """Calculates and stores the message it will send to the given link.
        Throws an error if it doesn't have the required messages."""
        msg = self.known_Belief
        for i, m in enumerate(self.msg_in):
            if i==link[0]:
                continue
            
            if m is None:
                # Set Undefined messages as [1,1]
                m = numpy.ones(len(msg.shape))
            
            msg = msg * m
        
        
        msg /= msg.sum()
        
        self.msg_out[link[0]] = msg


    def belief(self):
        """Returns the belief for this random variable, that is the marginal probability of
        the value of this random variable given all avaliable evidence."""
        ret = self.known_Belief
        
        for m in self.msg_in:
            if m is None:
                m =  [1,1]
            
            ret = ret * m
        
        ret /= ret.sum()
        
        return ret

    def set_known(self, value):
        self.known_Belief[0] = 1-value
        self.known_Belief[1] = value
        


class Unary:
    """Defines a unary factor, that is a function over a single random variable."""
    
    def __init__(self, dist, rv, rvl):
        """Constructed with the probability distribution over the random variable it is connected to,
        and the connection, as a random variable and the link of the random variable it is connected to."""
        self.dist = dist
        self.rv = rv
        self.rvl = rvl
    
    def send(self):
        """Makes it send it's message to the random variable it is connected to. link is provided for
        consistency with all other objects, but actually ignored."""
        self.rv.msg_in[self.rvl] = self.dist
        

class Factor:
    """Defines a Factor, which is a function over multiple Random Varaibles """
    
    def __init__(self,dist,links):
        self.dist = dist
        self.msg_in = [None] * links
        self.msg_out = [None] * links
    
    def calc_msg(self, link):
        
        msg = self.dist
        for i, m in enumerate(self.msg_in):
            if i==link[0]:
                continue
            
            if m is None:
                # Set Undefined messages as [1,1]
                m = [1,1]            
                
            #Reshpe message to match with the conditional
            proj = numpy.ones(len(msg.shape), int)
            proj[i] = -1
            m_proj = numpy.reshape(m,proj)
            
            msg = msg * m_proj
        
        #Marginalize/Sum 
        
        all_idx = list(range(len(msg.shape)))
        axes = tuple(all_idx[:(link[0])] + all_idx[(link[0]+1):])
        msg = msg.sum(axis=axes)
        
        msg /= msg.sum()
        
        self.msg_out[link[0]] = msg
        
        
class Edge:
    """Defines an Edge that connects Factors and Variables, Does message passing."""
    
    def __init__(self, rv0, rv0l, rv1, rv1l):
        "rvi- Variable/Factors connected To, rvil: The links in the Variable/Factors this edge slots into"
        # Record information...
        self.link0 = (rv0, rv0l)
        self.link1 = (rv1, rv1l)
       
       
    
    def send(self, link):
        """Sends the requested message down the link"""
        
        # Work out which way round we are going...
        if link==0:
            source = self.link1
            dest = self.link0
        
        else: # link==1
            source = self.link0
            dest = self.link1
            
        source[0].calc_msg([source[1]])
        # Fetch the message from the other side, make a duplicate so its not corrupted...
        other = source[0].msg_out[source[1]]
        if other is None:
            other = [1,1]
            
        # Send message...
        dest[0].msg_in[dest[1]] = other
        

       
    

#Define Random Variable and Add them to a List.
RVs = []
HE = RV(3)
RVs.append(HE)
FP = RV(3)
RVs.append(FP)
FC = RV(3)
RVs.append(FC)
WR = RV(2)
RVs.append(WR)
DP = RV(3)
RVs.append(DP)
FH = RV(2)
RVs.append(FH)
GS = RV(3)
RVs.append(GS)

PW = RV(3)
RVs.append(PW)
CB = RV(4)
RVs.append(CB)
GW = RV(4)
RVs.append(GW)
HW = RV(2)
RVs.append(HW)
LI = RV(1)
RVs.append(LI)

LS = RV(1)
RVs.append(LS)
VP = RV(1)
RVs.append(VP)
BS = RV(1)
RVs.append(BS)
LO = RV(1)
RVs.append(LO)
HP = RV(1)
RVs.append(HP)
ME = RV(2)
RVs.append(ME)
TA = RV(1)
RVs.append(TA)


#Define Uniaries
U_he = Unary(P_he, HE, 0)
U_fp = Unary(P_fp, FP, 0)
U_fc = Unary(P_fc, FC ,0)
U_wr = Unary(P_wr, WR, 0)
U_dp = Unary(P_dp, DP, 0)
U_gs = Unary(P_gs, GS, 0)
U_fh = Unary(P_fh, FH, 0)
                   
#Define Factors
F_ls_he = Factor(P_ls_he,2)
F_pw_he_fp = Factor(P_pw_he_fp, 3)
F_bs_fp_fc = Factor(P_bs_fp_fc, 3)
F_cb_pw_fc = Factor(P_cb_pw_fc, 3)
F_vp_pw = Factor(P_vp_pw, 2)
F_lo_cb = Factor(P_lo_cb, 2)
F_gw_cb_wr_dp = Factor(P_gw_cb_wr_dp, 4)
F_hw_gw_fh = Factor(P_hw_gw_fh, 3)
F_li_gw_gs = Factor(P_li_gw_gs, 3)
F_hp_cb_dp =Factor(P_hp_cb_dp,3)
F_me_gw_gs= Factor(P_me_gw_gs,3)
F_ta_me_hw = Factor(P_ta_me_hw,3)

#Define Edges Between Factors and Variables
Edges = []
Edges.append(Edge(HE,1,F_ls_he,1))
Edges.append(Edge(HE,2,F_pw_he_fp, 1))
Edges.append(Edge(LS,0,F_ls_he,0))

Edges.append(Edge(FP,2,F_pw_he_fp,2))
Edges.append(Edge(FP,1,F_bs_fp_fc,1))

Edges.append(Edge(FC,1,F_cb_pw_fc,2))
Edges.append(Edge(FC,2,F_bs_fp_fc,2))

Edges.append(Edge(BS,0,F_bs_fp_fc,0))

Edges.append(Edge(PW,0,F_pw_he_fp,0))
Edges.append(Edge(PW,1,F_vp_pw,1))
Edges.append(Edge(PW,2,F_cb_pw_fc,1))

Edges.append(Edge(VP,0,F_vp_pw,0))

Edges.append(Edge(CB,0,F_cb_pw_fc,0))
Edges.append(Edge(CB,1,F_gw_cb_wr_dp,1))
Edges.append(Edge(CB,2,F_lo_cb,1))
Edges.append(Edge(CB,3,F_hp_cb_dp,1))

Edges.append(Edge(LO,0,F_lo_cb,0))

Edges.append(Edge(WR,1,F_gw_cb_wr_dp,2))

Edges.append(Edge(GW,0,F_gw_cb_wr_dp,0))
Edges.append(Edge(GW,1,F_li_gw_gs,1))
Edges.append(Edge(GW,2,F_me_gw_gs,1))
Edges.append(Edge(GW,3,F_hw_gw_fh,1))

Edges.append(Edge(DP,1,F_gw_cb_wr_dp,3))
Edges.append(Edge(DP,2,F_hp_cb_dp,2))

Edges.append(Edge(HP,0,F_hp_cb_dp,0))

Edges.append(Edge(LI,0,F_li_gw_gs,0))

Edges.append(Edge(FH,1,F_hw_gw_fh,2))

Edges.append(Edge(GS,1,F_li_gw_gs,2))
Edges.append(Edge(GS,2,F_me_gw_gs,2))

Edges.append(Edge(ME,0,F_me_gw_gs,0))
Edges.append(Edge(ME,1,F_ta_me_hw,1))

Edges.append(Edge(HW,0,F_hw_gw_fh,0))
Edges.append(Edge(HW,1,F_ta_me_hw,2))

Edges.append(Edge(TA,0,F_ta_me_hw,0))

#Loopy Belief Propagation
def lbp(maxIter=50):
    
    ret = numpy.empty((len(RVs), 2))
    old_ret = numpy.empty((len(RVs), 2))
    
    #Send Unary
    U_he.send()
    U_fp.send()
    U_fc.send()
    U_wr.send()
    U_dp.send()
    U_gs.send()
    U_fh.send()
    
    curIter = 0
    Converged = False
    #Run Until Convergence or max Iterations reached
    while curIter < maxIter and not Converged:
        
        curIter += 1
        
        #Send to Factors from Variables
        for e in Edges:
            e.send(1)
            
        #Send From Factors to Variables
        for e in Edges:
            e.send(0)

        

        #Get new beliefs and check for convergence
        for row , rv in enumerate(RVs):
                ret[row,:] = rv.belief()

        if numpy.allclose(ret,old_ret): 
            Converged = True
        else:
            old_ret = numpy.copy(ret)
            

    return ret

    
 
   
def marginals(known):
    
    #Set test Results
    for r in RVs:
        r.set_known(0.5)
    for k, v in known.items():
        RVs[k].set_known(v)
        
    #Run loopy belief propagation    
    ret = lbp()
    
    return ret
 
#Note: Marginals returns the correct values the marginal probabilities except for P(ta)
    
def test_marginals(known):

    ret_brute = brute_marginals(known)  
    ret_lbp = marginals(known)

    for i, r in enumerate(RVs):
        print (numpy.allclose(ret_brute[i,:], ret_lbp[i,:]))
        

        
print('Marginals from running marginals({nti[ls]:0})') 
print(marginals({nti['ls']:0}))
        
print('Comparing marginals and brute_marginals with known = {} ')        
test_marginals({})

print('Comparing marginals and brute_marginals with known = {nti[me]:1}')        
test_marginals({nti['me'] : 1})
        
        

Marginals from running marginals({nti[ls]:0})
[[ 0.09476442  0.90523558]
 [ 0.99592972  0.00407028]
 [ 0.98017883  0.01982117]
 [ 0.10062027  0.89937973]
 [ 0.94974518  0.05025482]
 [ 0.99489212  0.00510788]
 [ 0.05112839  0.94887161]
 [ 0.09844898  0.90155102]
 [ 0.11631877  0.88368123]
 [ 0.24517579  0.75482421]
 [ 0.24903134  0.75096866]
 [ 0.92552277  0.07447723]
 [ 1.          0.        ]
 [ 0.10727906  0.89272094]
 [ 0.98593485  0.01406515]
 [ 0.11718046  0.88281954]
 [ 0.23303161  0.76696839]
 [ 0.35086263  0.64913737]
 [ 0.60901251  0.39098749]]
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
Comparing marginals and brute_marginals with known = {} 
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
Comparing marginals and brute_marginals with known = {nti[me]:1}
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


## 3. Bayesian Decision Theory

Bayesian decision theory sounds fancy, but is actually really simple. All you do is use Bayesian reasoning and a cost function to select the decision that, with everything integrated out, minimises the expected cost. In practise various approximations are usually made - here for instance we are using the mean model parameters instead of properly integrating them out; for a model such as this that is reasonable. Note that it isn't actually Bayesian unless you introduced a prior above!

Your task is to generate a flow chart indicating how a mechanic is to diagnose a problem with a coffee machine.
Here is an example: https://en.wikipedia.org/wiki/Flowchart#/media/File:LampFlowchart.svg Output it below the code block; don't worry about making it pretty but if your output is particularly hard to read feel free to copy & paste it into a markdown block and neaten it up.

Consider a machine to be diagnosed when confidence that one of the problems is in the broken state exceeds 90%. You can take multiple approaches to solving this problem, but the recommended technique:
 * Write a function to detect if a coffee machine is reliably diagnosed, given a dictionary of observed variables.
 * Write a recursive search function that takes a dictionary of observed variables. It will return a tree that includes the diagnostic variable to observe at each node and how long on average it takes to diagnose from that point in the tree.
 * Call the recursive search function with no observed variables and print out the tree it returns.
 
Note that this is very similar to constructing a random forest, but with a different objective, and instead of looping splits you loop unobserved diagnostic random variables. Remember to weight the machines by the probability of their defect occurring, which you learnt in step 1. Make sure you handle all observations being made and no defect being found (Don't delete these scenarios, as they do occur in real life; instead make sure that if everything is observed the defect is considered to have been found, even though the defect is probably with the user.).

__(10 marks)__
 * 2 marks for a function to detect if a machine is diagnosed.
 * 8 marks for the rest, whichever approach you choose to take.

In [46]:
# How long each diagnostic takes (in seconds)...
diag_time = dict()
diag_time[nti['ls']] = 20.0
diag_time[nti['vp']] = 240.0
diag_time[nti['bs']] = 3.0
diag_time[nti['lo']] = 3.0
diag_time[nti['hp']] = 10.0
diag_time[nti['me']] = 60.0
diag_time[nti['ta']] = 120.0

key_list = ['bs','lo','hp','ls', 'me','ta', 'vp']

# **************************************************************** 10 marks

def diagnostic(known,basic):
    
    ret = brute_marginals(known)
    index = -1
    threshold = 0.7  
    
    #Check if fault is diagnosed with 90% Confidence 
    #Take the fault with the highest probability 
    #Scale marginal by the chance of the fault happening and normalize 
    for i in range(0,7):
        scaled_ret = ret[i] * 1
        scaled_ret /= scaled_ret.sum()
        if i == 0 or i == 6 or i == 3:
            if scaled_ret[0] >= threshold:
                threshold = scaled_ret[0]
                index = i
        else:
            if scaled_ret[1] >= threshold:
                threshold = scaled_ret[1]
                index = i
            
    return index

#Finds the Best Test to run, given the known diagnostic test results
def find_split(known,basic):
    
    least_time = 300
    s_key =-1
    quick_key = -1
    #Choose Test that either finds a fault or take the least amount of time 
    for key in key_list:
        if nti[key] in known:
            continue
        else:
            known_new = dict(known)
            known_new[nti[key]] = 0
            new_fault_0 = diagnostic(known_new,basic)
            known_new[nti[key]] = 1
            new_fault_1 = diagnostic(known_new,basic)
            if new_fault_0 >= 0 or new_fault_1 >= 0:
                s_key = key
                
            else:
                if diag_time[nti[key]]  <= least_time:
                    least_time = diag_time[nti[key]]
                    quick_key = key
                    
    if s_key == -1:
        return quick_key
    else:
        return s_key
    
              
#Finds the Diagnostic Tree
def diag_tree(known,basic = marginals({})):
    fault = diagnostic(known,basic)
    if fault>=0:
        return {'leaf': True , 'fault' : fault , 'Time': 0 }
    elif len(list(known.keys())) == 7:
        return{'leaf': True,'fault': 'NO Fault Dected', 'Time': 0}
    else:
        split_var = find_split(known,basic)
        known0 = dict(known)
        known0[nti[split_var]] = 0
        l = diag_tree(known0,basic)
        known[nti[split_var]] = 1
        r = diag_tree(known,basic)
        return {'leaf': False ,
                'feature': split_var, 
                'left': l,
                'right': r,
                'Time': (l['Time']+r['Time'])/2 + diag_time[nti[split_var]]}
    
#Prints Tree
def print_tree(tree, indent = 0):
    if tree['leaf']:
        print(' '*indent, tree['fault'])
    else:

        print(' '*indent, '[' , tree['feature'],' = False', ' Tree Time = ', tree['left']['Time'], sep='' )
        print_tree(tree['left'], indent+4)
        print('')

        print(' '*indent, '[' , tree['feature'],' = True ', ' Tree Time = ', tree['right']['Time'], sep='')
        print_tree(tree['right'], indent+4)
        print('')
        
#Prints the Flowchart
def print_flowchart(known):
    
    tree = diag_tree(known)
    print('Average Time  of Tree = ', tree['Time'])
    print_tree(tree)
    
    
print_flowchart({})        



C:\Users\Lander\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in true_divide


 Average Time  of Tree =  153.25
[bs = False Tree Time = 300.5
    [lo = False Tree Time = 240.0
        [vp = False Tree Time = 0
             0

        [vp = True  Tree Time = 0
             2


    [lo = True  Tree Time = 355.0
        [hp = False Tree Time = 250.0
            [me = False Tree Time = 0
                 4

            [me = True  Tree Time = 380.0
                [ls = False Tree Time = 360.0
                    [ta = False Tree Time = 240.0
                        [vp = False Tree Time = 0
                             NO Fault Dected

                        [vp = True  Tree Time = 0
                             NO Fault Dected


                    [ta = True  Tree Time = 240.0
                        [vp = False Tree Time = 0
                             NO Fault Dected

                        [vp = True  Tree Time = 0
                             NO Fault Dected



                [ls = True  Tree Time = 360.0
                    [ta = False Tree Time = 240.0
 